<a href="https://colab.research.google.com/github/CristopherCano/Proyecto_Python_Procesamiento_de_datos/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Procesamiento de Datos con Python - BEDU - Módulo 3
## Viajes en Taxis y Ubers en la Zona Metropolitana del Valle de México 2016 - 2017
* Eduardo Alán Hernández Villasana
* Cristopher Arvizu Cano
* David Gil Peralta
* José Antonio Aguilar Téllez

## Planteamiento del Problema
En los últimos años, desde la llegada de aplicaciones para el transporte privado como Uber o Didi, los gremios taxistas de la Zona Metropolitana del Valle de México han protestado en contra de estas aplicaciones, ellos argumentan una competencia desleal y exigen regulaciones hacia estos transportes. Esto no es de extrañarse ya que, con el aumento de la popularidad de estas aplicaciones de transporte, los taxistas están perdiendo clientes y no pueden o quieren modernizarse para poder competir con los automóviles de Uber o Didi que son más modernos. 

A raíz de este enfrentamiento, han surgido noticias comparando estos dos modelos de transporte privado, sin embargo, muchas de estas noticias están sesgadas por intereses políticos y económicos. Es por esta razón que decidimos analizar datos recopilados tanto en Taxis y Ubers para hacer una comparación entre los dos modelos y aportar nuestros hallazgos estadísticos a la discusión de este problema.

## Preguntas Clave
Durante el desarrollo de este proyecto, se espera responder las siguientes preguntas.

* ¿Qué tipo de transporte pasa menos tiempo en el tráfico?
* ¿Cuál es la velocidad promedio de cada tipo de transporte?
* ¿Qué día de la semana hay mas demanda para cada tipo de transporte?
* ¿En qué tipo de transporte recibe más dinero en promedio por viaje?
* ¿Desde qué demsarcación territorial o municipio cada tipo de transporte tiene mayor demanda?
* ¿Cuál es el destino mas solicitado para cada tipo de transporte?

In [46]:
# Librerías a usar
import pandas as pd
import numpy as np

## Extracción de datos
Se descargará la base de datos desde GitHub.

In [47]:
url = "https://raw.githubusercontent.com/CristopherCano/Proyecto_Python_Procesamiento_de_datos/main/Data/cdmx_transporte_raw.csv"

In [48]:
data_raw = pd.read_csv(url, index_col=0)
data_raw.head(2)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,,
1,México DF Taxi de Sitio,2016-09-16 07:14:12,2016-09-18 04:41:40,-99.097369,19.416874,-99.202729,19.430353,N,120449,12373,242
2,México DF Taxi Libre,2016-09-18 06:16:33,2016-09-18 10:11:43,-99.297148,19.322128,-99.289949,19.326538,N,14110,1700,461


### Descripción de las Columnas
* **id**: Un identificador único para cada viaje.
* **vendor_id**: El tipo de transporte en que se realizó el viaje.
* **pickup_datetime**: Fecha y Hora en que el viaje inició.
* **dropoff_datetime**: Fecha y Hora en que el viaje finalizó.
* **pickup_longitude**: Longitud en la que el viaje inició.
* **pickup_latitude**: Latitud en la que el viaje inició.
* **dropoff_longitude**: Longitud en la que el viaje finalizó.
* **dropoff_latitude**: Latitud en la que el viaje finalizó.
* **store_and_fwd_flag**: Indica si la información del viaje se almacenó en una memoria y después se envio al servidor por que no tenía conexión a internet o no. Todas las entradas de datos estan en 'N'.
* **trip_duration**: Duración del viaje en segundos.
* **dist_meters**: Distancia recorrida en el viaje en metros.
* **wait_sec**: Tiempo en segundos en el que el automovil estuvo completamente detenido durante el viaje. (Se usará como medición del tráfico.)

## Exploración básica de datos
### Hallazgos preliminares encontraros 

1. ¿El conjunto de datos que tengo realmente me sirve para responder algunas de las preguntas que me planteé?

In [49]:
tamaño = data_raw.shape
print(f'2. ¿Qué tamaño tiene mi conjunto de datos? \n número de filas: {tamaño[0]} \n número de columnas: {tamaño[1]}')

2. ¿Qué tamaño tiene mi conjunto de datos? 
 número de filas: 12694 
 número de columnas: 11


In [50]:
print(f'3. ¿Qué columnas tengo y qué información tengo en cada una de esas columnas? \n \n En total tenemos{tamaño[1]} columnas \n \n {data_raw.columns}')

3. ¿Qué columnas tengo y qué información tengo en cada una de esas columnas? 
 
 En total tenemos11 columnas 
 
 Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'store_and_fwd_flag', 'trip_duration', 'dist_meters', 'wait_sec'],
      dtype='object')


4. Los nombres que tienen mis columnas, ¿son el nombre más apropiado?

Podemos observar que algunos nombres como vendor_id o dist_meters no son los más adecuados por lo que tendremos que usar la conveción apropiada que es snake-case y ajustar sus nombres

5. ¿Qué tipos de datos tengo en cada columna? ¿Parecen ser el tipo correcto de datos? ¿O es un tipo de datos "incorrecto"?



In [51]:
data_raw.dtypes

vendor_id              object
pickup_datetime        object
dropoff_datetime       object
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dist_meters             int64
wait_sec                int64
dtype: object

Las fechas aparecen como object por lo que sera necesario transformar esta variable a un formato de fecha mediante padas

6. Si selecciono algunas filas al azar y las observo, ¿estoy obteniendo los datos que debería? ¿O hay datos que parecen estar "sucios" o "incorrectos"?


In [52]:
data_raw.sample(5)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,,
11792,México DF Taxi Libre,2017-07-10 09:34:39,2017-07-10 09:39:21,-99.196463,19.461485,-99.186452,19.457158,N,283,1935,63
3739,México DF Taxi Libre,2017-05-08 07:43:06,2017-05-08 07:57:37,-99.213136,19.482802,-99.193726,19.463996,N,871,5665,159
2234,México DF Taxi de Sitio,2017-04-14 01:59:51,2017-04-14 02:30:50,-99.264151,19.330830,-99.232517,19.366740,N,1859,13113,54
2605,México DF Radio Taxi,2016-10-30 10:41:49,2016-10-31 02:09:06,-99.182057,19.368907,-99.170277,19.399965,N,55637,11195,979
741,México DF Taxi Libre,2016-07-07 06:28:19,2016-07-07 06:35:36,-99.205022,19.366841,-99.214870,19.354766,N,438,2580,110


A continuación observamos los siguiente problemas:
* **vendor_id**: Se repite el patrón México DF
* **store_and_fwd_flag**: No es significativa esta variable ya que siempre marca "N" en cada fila
* **trip_duration**: Tenemos duraciones de viaje exageradamente grandes
* **dist_meters**: Tenemos distacias de viaje exageradamente grandes
* **wait_sec**: Tenemos tiempos de espera de viaje exageradamente grandes

## Limpieza de datos y agregaciones

In [53]:
# Aqui va la corrección de nombres de las columnas

In [54]:
data_raw_dropped = data_raw.drop(['store_and_fwd_flag'], axis=1)

In [55]:
 data_raw_dropped['vendor_id'] = data_raw_dropped['vendor_id'].str.replace("México DF ","")

In [56]:
data_raw_dropped.head(10)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,
1,Taxi de Sitio,2016-09-16 07:14:12,2016-09-18 04:41:40,-99.097369,19.416874,-99.202729,19.430353,120449,12373,242
2,Taxi Libre,2016-09-18 06:16:33,2016-09-18 10:11:43,-99.297148,19.322128,-99.289949,19.326538,14110,1700,461
3,Taxi Libre,2016-09-18 10:11:50,2016-09-18 10:23:11,-99.289603,19.326263,-99.271874,19.328530,681,2848,129
4,Taxi Libre,2016-09-18 10:23:38,2016-09-18 10:30:53,-99.271161,19.328875,-99.279900,19.326256,436,1409,106
5,Taxi Libre,2016-09-18 10:44:18,2016-09-18 10:51:40,-99.282761,19.326944,-99.291705,19.322754,442,1567,85
6,Taxi Libre,2016-09-18 10:58:07,2016-09-18 10:59:46,-99.289712,19.322929,-99.289280,19.325978,100,797,19
7,Taxi Libre,2016-09-18 11:00:00,2016-09-18 11:05:45,-99.285391,19.326247,-99.282549,19.328117,345,676,169
8,Taxi Libre,2016-09-18 11:08:03,2016-09-18 11:17:06,-99.285891,19.330022,-99.289828,19.319170,544,3771,37
9,Radio Taxi,2016-09-18 01:06:08,2016-09-18 01:26:34,-99.182346,19.370885,-99.176534,19.341410,1226,5662,572


In [57]:
data_raw_dropped['pickup_datetime'] = pd.to_datetime(data_raw_dropped['pickup_datetime'], unit = 'ns')
data_raw_dropped['pickup_datetime'].head()

id
1   2016-09-16 07:14:12
2   2016-09-18 06:16:33
3   2016-09-18 10:11:50
4   2016-09-18 10:23:38
5   2016-09-18 10:44:18
Name: pickup_datetime, dtype: datetime64[ns]

In [58]:
data_raw_dropped['dropoff_datetime'] = pd.to_datetime(data_raw_dropped['dropoff_datetime'], unit = 'ns')
data_raw_dropped['dropoff_datetime'].head()

id
1   2016-09-18 04:41:40
2   2016-09-18 10:11:43
3   2016-09-18 10:23:11
4   2016-09-18 10:30:53
5   2016-09-18 10:51:40
Name: dropoff_datetime, dtype: datetime64[ns]

In [59]:
data_raw_dropped.dtypes

vendor_id                    object
pickup_datetime      datetime64[ns]
dropoff_datetime     datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
trip_duration                 int64
dist_meters                   int64
wait_sec                      int64
dtype: object

### Se han encontrado algunos viajes con velocidades, distacias o diraciones irreales, esto puede ser debido a errores en la recolección de los datos. Posteriormente se eliminatán los outliers de estas columnas.
Para hacer el filtrado más facil, se convertiran las distancias de metros a kilómetros, la duración de los viajes de segundos a horas y se deducirán las velocidades en kilómetros por hora.

In [60]:
data_filtered = data_raw_dropped.copy()

Conversión de la duración del viaje de segundos a horas.

In [61]:
data_filtered["trip_duration"].head(2)

id
1    120449
2     14110
Name: trip_duration, dtype: int64

In [62]:
# Como es una operación simple se usará una función anónima.
data_filtered["trip_duration"] = data_filtered["trip_duration"].apply(lambda x: x/3600)
data_filtered["trip_duration"].head(2)

id
1    33.458056
2     3.919444
Name: trip_duration, dtype: float64

Conversión de la distancia recorrida en metros a kilómetros

In [63]:
data_filtered["dist_meters"].head(2)

id
1    12373
2     1700
Name: dist_meters, dtype: int64

In [64]:
# Como es una operación simple se usará una función anónima.
data_filtered["dist_km"] = data_filtered["dist_meters"].apply(lambda x: x/1000)
data_filtered["dist_km"].head(2)

id
1    12.373
2     1.700
Name: dist_km, dtype: float64

Creación de la columna `speed_km` que contiene la velocidad del viaje.

In [65]:
data_filtered["speed_km"] = data_filtered["dist_km"]/data_filtered["trip_duration"]
data_filtered[["dist_km","trip_duration","speed_km"]].head()

,dist_km,trip_duration,speed_km
id,,,
1,12.373,33.458056,0.369806
2,1.700,3.919444,0.433735
3,2.848,0.189167,15.055507
4,1.409,0.121111,11.633945
5,1.567,0.122778,12.762896


Como se puede ver, hay viajes que duran más de lo anormal, debido al contexto del dataframe, consideramos que se deben de eliminar los viajes con velocidades mayores a 200 km/h y con distancias menores a 500 metros (.5 km). Después se eliminarán outliers con el método del Rango Inter Cuartílico.

In [66]:
data_filtered = data_filtered[(data_filtered["speed_km"] < 200)  & (data_filtered["dist_km"] > 0.5)]
data_filtered.shape

(11411, 12)

Eliminación de Outliers de `speed_km`, `dist_km`, `trip_duration`

In [67]:
Q1 = data_filtered["speed_km"].quantile(0.25)
Q2 = data_filtered["speed_km"].quantile(0.5)
Q3 = data_filtered["speed_km"].quantile(0.75)
IQR_speed = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_speed)

Cuartiles		Valor
25%		 9.185913076398961
50%		 15.399341021416804
75%		 20.984072667115985


Rango Intercuartílico:  11.798159590717024


In [68]:
data_filtered = data_filtered[(data_filtered["speed_km"] >= Q1 - 1.5*IQR_speed) & (data_filtered["speed_km"] <= Q3 + 1.5*IQR_speed)]

In [69]:
Q1 = data_filtered["dist_km"].quantile(0.25)
Q2 = data_filtered["dist_km"].quantile(0.5)
Q3 = data_filtered["dist_km"].quantile(0.75)
IQR_dist = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_dist)

Cuartiles		Valor
25%		 2.35
50%		 4.428
75%		 8.7295


Rango Intercuartílico:  6.3795


In [70]:
data_filtered = data_filtered[(data_filtered["dist_km"] >= Q1 - 1.5*IQR_dist) & (data_filtered["dist_km"] <= Q3 + 1.5*IQR_dist)]

In [71]:
Q1 = data_filtered["trip_duration"].quantile(0.25)
Q2 = data_filtered["trip_duration"].quantile(0.5)
Q3 = data_filtered["trip_duration"].quantile(0.75)
IQR_trip_duration = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_trip_duration)

Cuartiles		Valor
25%		 0.15527777777777776
50%		 0.2936111111111111
75%		 0.587013888888889


Rango Intercuartílico:  0.4317361111111112


In [72]:
data_filtered = data_filtered[(data_filtered["trip_duration"] >= Q1 - 1.5*IQR_trip_duration) & (data_filtered["trip_duration"] <= Q3 + 1.5*IQR_trip_duration)]

In [73]:
data_filtered.shape

(8937, 12)

## Costos

In [74]:
url_tarifas = "https://raw.githubusercontent.com/CristopherCano/Proyecto_Python_Procesamiento_de_datos/main/Data/tarifas.json"

tarifas = pd.read_json(url_tarifas)
tarifas

,Transporte,Banderazo,Tarifa Distancia,Tarifa Tiempo,Tarifa Mínima
0,Taxi Libre,8.74,1.07,1.07,8.74
1,Taxi de Sitio,13.10,1.30,1.30,13.10
2,Radio Taxi,27.60,1.84,1.84,27.60
3,UberX,7.00,3.57,1.80,35.00
4,UberXL,12.15,6.28,3.15,45.50
5,UberBlack,30.00,9.46,3.50,77.00
6,UberSUV,40.00,15.00,4.00,150.00


In [75]:
tarifas.dtypes

Transporte           object
Banderazo           float64
Tarifa Distancia    float64
Tarifa Tiempo       float64
Tarifa Mínima       float64
dtype: object

In [76]:
tarifas.columns = ["transporte", "banderazo", "tarifa_dist", "tarifa_tiempo", "tarifa_min"]
tarifas


,transporte,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min
0,Taxi Libre,8.74,1.07,1.07,8.74
1,Taxi de Sitio,13.10,1.30,1.30,13.10
2,Radio Taxi,27.60,1.84,1.84,27.60
3,UberX,7.00,3.57,1.80,35.00
4,UberXL,12.15,6.28,3.15,45.50
5,UberBlack,30.00,9.46,3.50,77.00
6,UberSUV,40.00,15.00,4.00,150.00


In [121]:
viajes = data_filtered[["vendor_id", "wait_sec", "dist_meters"]]

costos_viajes = pd.merge(viajes, tarifas, left_on="vendor_id", right_on="transporte", how="left")
costos_viajes = costos_viajes.drop(columns=["transporte"])
costos_viajes

,vendor_id,wait_sec,dist_meters,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min
0,Taxi Libre,129,2848,8.74,1.07,1.07,8.74
1,Taxi Libre,106,1409,8.74,1.07,1.07,8.74
2,Taxi Libre,85,1567,8.74,1.07,1.07,8.74
3,Taxi Libre,19,797,8.74,1.07,1.07,8.74
4,Taxi Libre,169,676,8.74,1.07,1.07,8.74
...,...,...,...,...,...,...,...
8932,Taxi Libre,386,7407,8.74,1.07,1.07,8.74
8933,Taxi Libre,33,2280,8.74,1.07,1.07,8.74
8934,Radio Taxi,427,8261,27.60,1.84,1.84,27.60
8935,Taxi de Sitio,115,2498,13.10,1.30,1.30,13.10


In [122]:
costos_viajes.loc[(costos_viajes["vendor_id"] == "Taxi Libre") |
                  (costos_viajes["vendor_id"] == "Taxi de Sitio") |
                  (costos_viajes["vendor_id"] == "Radio Taxi"),
                  "costo_viaje_sin_min"] = (costos_viajes["banderazo"] + 
                                ((costos_viajes["dist_meters"] / 250) * costos_viajes["tarifa_dist"]) + 
                                ((costos_viajes["wait_sec"] / 45) * costos_viajes["tarifa_tiempo"]))


costos_viajes.loc[(costos_viajes["vendor_id"] == "UberX") |
                  (costos_viajes["vendor_id"] == "UberXL") |
                  (costos_viajes["vendor_id"] == "UberBlack") |
                  (costos_viajes["vendor_id"] == "UberSUV"),
                  "costo_viaje_sin_min"] = (costos_viajes["banderazo"] + 
                                ((costos_viajes["dist_meters"] / 1000) * costos_viajes["tarifa_dist"]) + 
                                ((costos_viajes["wait_sec"] / 60) * costos_viajes["tarifa_tiempo"]))

costos_viajes

,vendor_id,wait_sec,dist_meters,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min,costo_viaje_sin_min
0,Taxi Libre,129,2848,8.74,1.07,1.07,8.74,23.996773
1,Taxi Libre,106,1409,8.74,1.07,1.07,8.74,17.290964
2,Taxi Libre,85,1567,8.74,1.07,1.07,8.74,17.467871
3,Taxi Libre,19,797,8.74,1.07,1.07,8.74,12.602938
4,Taxi Libre,169,676,8.74,1.07,1.07,8.74,15.651724
...,...,...,...,...,...,...,...,...
8932,Taxi Libre,386,7407,8.74,1.07,1.07,8.74,49.620182
8933,Taxi Libre,33,2280,8.74,1.07,1.07,8.74,19.283067
8934,Radio Taxi,427,8261,27.60,1.84,1.84,27.60,105.860516
8935,Taxi de Sitio,115,2498,13.10,1.30,1.30,13.10,29.411822


In [127]:
costos_viajes["costo_viaje_sobre_min"] = costos_viajes["costo_viaje_sin_min"]

costos_viajes.loc[costos_viajes["costo_viaje_sobre_min"] < costos_viajes["tarifa_min"], "costo_viaje_sobre_min"] = costos_viajes["tarifa_min"]

In [128]:
costos_viajes[costos_viajes["vendor_id"] == "UberBlack"]

,vendor_id,wait_sec,dist_meters,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min,costo_viaje_sin_min,costo_viaje_sobre_min
94,UberBlack,40,9061,30.0,9.46,3.5,77.0,118.050393,118.050393
489,UberBlack,20,2788,30.0,9.46,3.5,77.0,57.541147,77.000000
1116,UberBlack,3643,1857,30.0,9.46,3.5,77.0,260.075553,260.075553
1117,UberBlack,4656,4261,30.0,9.46,3.5,77.0,341.909060,341.909060
1119,UberBlack,5443,3769,30.0,9.46,3.5,77.0,383.163073,383.163073
2058,UberBlack,207,978,30.0,9.46,3.5,77.0,51.326880,77.000000
2059,UberBlack,68,594,30.0,9.46,3.5,77.0,39.585907,77.000000
3375,UberBlack,189,3345,30.0,9.46,3.5,77.0,72.668700,77.000000
3777,UberBlack,1028,15061,30.0,9.46,3.5,77.0,232.443727,232.443727
5201,UberBlack,211,7756,30.0,9.46,3.5,77.0,115.680093,115.680093
